In [1]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
def make_urllist(date):
    urllist = []
    page_num = 1
    last_urls = set()

    while True:
        url = f'https://finance.naver.com/news/mainnews.naver?date={date}&page={page_num}'
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
        news = requests.get(url, headers=headers)

        # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
        soup = BeautifulSoup(news.content, 'html.parser')

        # CASE 1
        news_list = soup.select('li dl')
        # CASE 2
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))

        current_urls = set()

        # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
        for line in news_list:
            news_url = line.a.get('href')
            current_urls.add(news_url)

        # 현재 페이지의 URL 목록이 이전 페이지와 동일하면 종료합니다.
        if current_urls == last_urls:
            break

        # 중복을 확인한 후에 URL을 리스트에 추가합니다.
        urllist.extend(current_urls)
        last_urls = current_urls
        page_num += 1

    return urllist

In [44]:
original_urls = make_urllist('2024-07-22')
print('뉴스 기사의 개수: ',len(original_urls))

뉴스 기사의 개수:  199


In [45]:
# 변환된 URL을 담을 리스트
url_list = []

for url in original_urls:
    # URL에서 article_id와 office_id 추출
    article_id = url.split('article_id=')[1].split('&')[0]
    office_id = url.split('office_id=')[1].split('&')[0]

    # 새로운 형식의 URL 생성
    new_url = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
    
    # 리스트에 추가
    url_list.append(new_url)

# 결과 출력
print(url_list)

['https://n.news.naver.com/mnews/article/648/0000027527', 'https://n.news.naver.com/mnews/article/366/0001006598', 'https://n.news.naver.com/mnews/article/243/0000062264', 'https://n.news.naver.com/mnews/article/277/0005449146', 'https://n.news.naver.com/mnews/article/011/0004370105', 'https://n.news.naver.com/mnews/article/014/0005217032', 'https://n.news.naver.com/mnews/article/029/0002889592', 'https://n.news.naver.com/mnews/article/009/0005338815', 'https://n.news.naver.com/mnews/article/009/0005338784', 'https://n.news.naver.com/mnews/article/029/0002889577', 'https://n.news.naver.com/mnews/article/014/0005217010', 'https://n.news.naver.com/mnews/article/014/0005217034', 'https://n.news.naver.com/mnews/article/057/0001831301', 'https://n.news.naver.com/mnews/article/029/0002889578', 'https://n.news.naver.com/mnews/article/009/0005338847', 'https://n.news.naver.com/mnews/article/031/0000855494', 'https://n.news.naver.com/mnews/article/215/0001171963', 'https://n.news.naver.com/mnew

In [46]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [47]:
#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code'] = idx2word[str(code)]
  return df

In [48]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [49]:
news_titles = []
news_url =[]
news_contents =[]
news_dates = []

In [50]:
for i in tqdm(url_list):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",len(url_list),'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(url_list)
print("\n[뉴스 내용]")
print(news_contents)

100%|██████████| 199/199 [03:29<00:00,  1.05s/it]

검색된 기사 갯수: 총  199 개

[뉴스 제목]
['中 3중전회 후 엇갈린 ETF…과창판 날고 H지수 뒷걸음질[ETF워치]', '두산로보틱스와 합병하는 두산밥캣, 오는 11월 자사주 소각 방침', '‘바이든 사퇴’ 여파…코스피 2760선 후퇴', '뉴욕증시, 기술주 랠리에 상승…6월 PCE·2분기 성장률 주목', 'M7 조정에도 살아 남은 애플…개미들 이달에만 3000억 원 순매수', '아! 코스닥… 상장사 68% 올해 주가 빠졌다', '기술주 발작 끝?...테슬라·엔비디아 동반 반등', '“밸류업 아니라 밸류킬이네”…현금청산 당하는 개미들, 올 증시 9조 ‘땡처리’', '배당 안주고 주가 누르다 상폐 … 헐값에 현금청산 내몰린 개미', '[기획] 美대선 격랑… 세계경제 `시계 제로`', '부동산 신탁사 13곳 합쳐 순익 269억… 1년새 6분의 1 토막', "'밸류업' 작전 변경… 금융 담고 車 덜어내는 기관", '강남·마용성 뛰는데 지방은 "손님 없어요"…서울 안에서도 양극화', '운용사, 커버드콜 ETF 전쟁… `리브랜딩` 부스트', '배당 안주고 주가 누르더니 결국 상폐…헐값에 현금청산, 개미들 피눈물', '논란의 두산그룹 지배구조 개편…"특별관계자 의결권 제한해야"', "'바이든 쇼크' 2760선 후퇴...외국인 또 매도", '[마켓인]NF3 파는 효성화학…베트남법인·필름사업부도 매각할까', '김병환, 삼부토건 주가조작설에 "판단 일러"…두산 합병엔 "제도 점검" [종합]', '[바이든 후보 사퇴] 美 대선판 요동… 금융시장, 트럼프 트레이드 약화?', "배터리株 '트럼프 리스크'…코스닥 800 방어선 위태", "엔화값 반등에…엔화 노출 美ETF도 '햇살'", '"매달 통장에 월급처럼 꽂혀요"…주식 다 팔고 갈아탔다', '\'트럼프 트레이드\' 여전…"美 대선까지 롤러코스터 가능성"', '모건스탠리, SK하이닉스 빼고 아모레 담았다', '힘 좋은 건·조·기…코스피 흔들려도 굳건', '한투, 카뱅 최대주주 되나...시나리오 검토 착수', "공모주 열

In [51]:
df5 = pd.DataFrame({
    'news_title':news_titles,
    'news_url': url_list,
    'news_contents': news_contents,
    'news_dates': news_dates
})

In [52]:
df1

,news_title,news_url,news_contents,news_dates
0,줍줍까지 나왔는데 웃돈만 10억…'얼죽신' 열풍에 입주권 귀하신 몸,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 ...,2024-07-26 20:06:12
1,‘AI 반도체’ 리벨리온 IPO 주관사에 대표 삼성·공동 한투 선정,https://n.news.naver.com/mnews/article/366/000...,[\n기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 ...,2024-07-26 17:46:14
2,서울 세운5-1·3구역 대출 연장 성공…규모 1000억 확대,https://n.news.naver.com/mnews/article/018/000...,[\n대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에...,2024-07-26 19:48:07
3,"배임까지… 두산밥캣, 6거래일째 하락 이어 시간外 약세",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹...,2024-07-26 17:21:13
4,상승 출발 엔비디아 vs 하락 출발 테슬라 …과연 누가 웃을까?,https://n.news.naver.com/mnews/article/029/000...,[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자...,2024-07-26 22:56:07
...,...,...,...,...
142,“실적 좋은데 주가 왜이래”…美 빅테크 ‘찬물’ 삼전·하이닉스 강타,https://n.news.naver.com/mnews/article/018/000...,"[\n'깜짝 실적' 무색…'20만닉스' 깨지고, '8만전자' 위태美 빅테크 급락 영...",2024-07-26 05:06:12
143,[속보]나스닥·S&amp;P500 반등 실패…러셀지수 1.34%↑,https://n.news.naver.com/mnews/article/018/000...,[\n\t\t\t[뉴욕=이데일리 김상윤 특파원] 전날 급락했던 나스닥 지수와 S&a...,2024-07-26 05:14:20
144,"""이틀간 9930억원 팔아치웠다"" 증시, 외국인 엑소더스 이어진다?",https://n.news.naver.com/mnews/article/417/000...,"[\n미국 대선, M7 실적 발표 등이 영향\n\n\n\n대외 변수로 외국인 투자자...",2024-07-26 05:11:00
145,삼전 말고 삼중·삼바로…눈돌리는 외국인·기관,https://n.news.naver.com/mnews/article/018/000...,[\n외국인 일주일간 삼전 2693억 순매도…기관도 5032억 팔아HBM 테스트 지...,2024-07-26 05:01:14


In [53]:
df2

,news_title,news_url,news_contents,news_dates
0,“이달에만 400억원 뭉칫돈”…일학개미 ‘이 상품’에 꽂혔다,https://n.news.naver.com/mnews/article/009/000...,[\n엔화 강세·美 금리인하 겹치자채권값 올라 고수익 기대 커져20년 이상 초장기채...,2024-07-25 22:02:14
1,"빗썸, 업비트에 바짝… 무료수수료 효과?",https://n.news.naver.com/mnews/article/029/000...,"[\n점유율 격차, 지난 3월 50%p→6월 31.3%p 축소""빗썸 상장 및 수수료...",2024-07-25 18:25:09
2,"외국인, 반도체·차 팔고 덜 오른 종목 갈아탔다",https://n.news.naver.com/mnews/article/014/000...,[\n17일부터 코스피 1조8024억 매도삼성重· LG전자·한화에어로 매수 \n\n...,2024-07-25 18:28:17
3,[뉴스추적] 티몬·위메프 사태 일파만파…피해 걷잡을 수 없이 커져,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 피해는 걷잡을 수 없이 커지고 있습니다. 자세...,2024-07-25 19:22:14
4,"뉴욕증시, 기술주 재평가＋경제지표·기업실적 주목…혼조 출발",https://n.news.naver.com/mnews/article/001/001...,[\n\n\n\n\n뉴욕증권거래소(AFP=연합뉴스 자료사진)(뉴욕=연합뉴스) 김 현...,2024-07-25 23:53:11
...,...,...,...,...
205,"테슬라에 IB혹평 ""로보택시도 실체 없다""…주가 12.3% 폭락(종합)",https://n.news.naver.com/mnews/article/018/000...,"[\n연중 수익률 다시 마이너스…-13.05%""이제 초점, 다시 펀더멘털로 돌아가야...",2024-07-25 02:36:15
206,동전주 띄워 개미 등친 '작전'…찾기도 잡기도 힘든 해외로,https://n.news.naver.com/mnews/article/008/000...,"[\n[홍콩 리딩방, 끝나지 않는 작전]③[편집자주] 국내에서 대거 매수한 해외 동...",2024-07-25 05:00:00
207,"ELS 충격 털어낸 KB금융, 목표가 줄줄이 상향",https://n.news.naver.com/mnews/article/018/000...,[\n실적 발표 후 14개 증권사 목표주가 상향사상 최대 실적에 11만6000원까지...,2024-07-25 05:15:11
208,미 빅테크주 급락 `쇼크`…돈만 드는 AI? `구글 5%·엔비디아6%` 뚝,https://n.news.naver.com/mnews/article/029/000...,[\n필라델피아 반도체지수 5% 하락\n\n\n\n[연합뉴스 제공] 미국 빅테...,2024-07-25 05:05:08


In [54]:
df3

,news_title,news_url,news_contents,news_dates
0,어닝 쇼크까지…빅테크 주가 '폭풍 속으로',https://n.news.naver.com/mnews/article/015/000...,"[\n희비 갈린 2분기 성적표테슬라, 車 판매 '급브레이크'작년보다 순이익 45% ...",2024-07-24 17:48:17
1,"PF부실, 저축銀·캐피털 신용도 강타",https://n.news.naver.com/mnews/article/009/000...,[\n상반기 신평사 신용평가 결과금융업종 하향 비중 역대 최대PF 위험노출 큰 2금...,2024-07-24 17:49:08
2,주주 반발에…당국 두산합병 정정보고서 요구,https://n.news.naver.com/mnews/article/011/000...,[\n두산 합병 증권신고서 효력 정지\n\n\n\n[서울경제] 금융감독원이 두산로보...,2024-07-24 18:10:10
3,큰손 쓸어담은 '스타 채권' 2년새 30% 수익,https://n.news.naver.com/mnews/article/009/000...,[\n금리 상승시기 가격 급락 틈타액면금리 1%대 저쿠폰채 매수시중금리 떨어지자 대...,2024-07-24 17:51:18
4,"비트코인, `트럼프 입` 주목…이번 주말 변곡점 되나",https://n.news.naver.com/mnews/article/029/000...,[\n\n\n\n\n[픽사베이 제공] 가상화폐 대장주 비트코인이 6만6000달...,2024-07-24 18:35:10
...,...,...,...,...
161,美 불확실성에 갈팡질팡 外人…조선·방산株는 담았다,https://n.news.naver.com/mnews/article/018/000...,"[\n美 대선 혼란 속…외국인, 5거래일 만에 순매수 전환불확실성 커지자 위험 자산...",2024-07-24 00:01:13
162,[STOCK] '날개없는 추락' 카카오페이 김범수 구속에 8% 급락… 신사업 제동,https://n.news.naver.com/mnews/article/417/000...,[\n\n\n\n\n SM엔터테인먼트 시세조종 의혹을 받는 김범수 카카오 경영쇄신위...,2024-07-24 05:08:00
163,올해 140% 뛰었는데 증여까지…분주한 한미반도체 오너家[마켓인],https://n.news.naver.com/mnews/article/018/000...,"[\n오너 2세 곽동신 부회장, 두 아들에 증여작년 446% 뛸 때도 지분 1%씩 ...",2024-07-24 04:23:10
164,오르긴 올랐는데 더 갈 수 있을까…혼돈의 반도체株,https://n.news.naver.com/mnews/article/018/000...,"[\n삼성전자, 외국인 4일만의 순매수 속 1% 상승삼성전자 오르며 코스피도 5거래...",2024-07-24 05:01:14


In [55]:
df4

,news_title,news_url,news_contents,news_dates
0,두산 지배구조 개편에… 외국인도 주식 팔고 떠났다,https://n.news.naver.com/mnews/article/005/000...,[\n에너빌리티·밥캣 주식 2400억원 순매도\n\n\n\n두산그룹 지배구조 개편 ...,2024-07-23 18:17:08
1,두산그룹 합병 논란에 재부상한 '이사충실의무',https://n.news.naver.com/mnews/article/648/000...,"[\n민주당 23일 '주주의 비례적 이익과 밸류업' 토론회""기업 M&amp;A, 일...",2024-07-23 18:15:09
2,코스피 5일만에 반등…반도체 끌고 실적주 밀고,https://n.news.naver.com/mnews/article/277/000...,[\n\n\n\n\n코스피가 23일 반도체주와 HD현대일렉트릭 등 실적 주도주의 강...,2024-07-23 18:13:09
3,[마켓인]“신용도 하향 기조 심화…하반기 변수도 ‘PF’”,https://n.news.naver.com/mnews/article/018/000...,[\n한신평 ‘2024년 상반기 정기평가 결과와 하반기 산업별 전망’[이데일리 마켓...,2024-07-23 18:26:18
4,'깜짝실적' 대형주의 반격… 2900선 고지 탈환 앞장선다,https://n.news.naver.com/mnews/article/014/000...,[\n2분기 실적시즌 본격화SK하이닉스 최대 매출 전망현대차·기아 영업익 8조 기대...,2024-07-23 18:22:17
...,...,...,...,...
200,"[마켓인]""펀드도 결성했는데""...엔케이에스인베, 자본잠식 '경고'",https://n.news.naver.com/mnews/article/018/000...,[\n신규 VC 정점 찍은 2022년도 태동지난해 9월 20억원 규모 펀드 결성자본...,2024-07-23 04:58:13
201,"""개미들 떨고있니"" 증시 불안에 갈곳 잃은 자금… 상승 동력은?",https://n.news.naver.com/mnews/article/417/000...,[\n\n\n\n\n국내 증시가 불안정한 모습을 보이자 증시 대기성 자금이 늘어나는...,2024-07-23 04:53:00
202,트럼프에 공장 건설 중단까지…위기의 2차전지주,https://n.news.naver.com/mnews/article/018/000...,"[\nLG엔솔, 전 날보다 4.92% 내려…에코프로비엠 6%↓트럼프 ""취임 첫날 전...",2024-07-23 00:01:19
203,"뉴욕증시, 바이든 재선 포기 후 첫 거래일 기술주↑…상승 출발",https://n.news.naver.com/mnews/article/001/001...,[\n\n\n\n\n뉴욕증권거래소(UPI=연합뉴스 자료사진)(뉴욕=연합뉴스) 김 현...,2024-07-23 00:01:44


In [56]:
df5

,news_title,news_url,news_contents,news_dates
0,中 3중전회 후 엇갈린 ETF…과창판 날고 H지수 뒷걸음질[ETF워치],https://n.news.naver.com/mnews/article/648/000...,[\n중국 반도체주 강세…빅테크 영향 홍콩 약세'트럼프 트레이드' 영향…2차전지·T...,2024-07-22 18:00:10
1,"두산로보틱스와 합병하는 두산밥캣, 오는 11월 자사주 소각 방침",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣 로고. /두산밥캣 제공 두산밥캣이 기존 자...,2024-07-22 20:24:13
2,‘바이든 사퇴’ 여파…코스피 2760선 후퇴,https://n.news.naver.com/mnews/article/243/000...,"[\n바이든, SNS에 “물러날 것…해리스 지지”韓 증시 여파…반도체·이차전지주 흔...",2024-07-22 20:55:08
3,"뉴욕증시, 기술주 랠리에 상승…6월 PCE·2분기 성장률 주목",https://n.news.naver.com/mnews/article/277/000...,"[\n바이든 재선 포기…'트럼프 트레이드' 진정되나25일 2분기 GDP, 26일 6...",2024-07-22 22:50:26
4,M7 조정에도 살아 남은 애플…개미들 이달에만 3000억 원 순매수,https://n.news.naver.com/mnews/article/011/000...,[\n[서학개미 리포트]中판매 회복에 AI 아이폰 기대감印시장도 공략···주가 상승...,2024-07-22 18:02:10
...,...,...,...,...
194,"셀트리온, 하반기 짐펜트라 잠재력 확인-상상인",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반...,2024-07-22 05:09:12
195,"열기 빠진 시장…피앤에스미캐닉스 VS 아이빔테크놀로지, 증권가 ‘픽’은",https://n.news.naver.com/mnews/article/018/000...,"[\n[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호...",2024-07-22 00:01:09
196,5대 금융 트래블 체크카드 가입자 650만명 육박,https://n.news.naver.com/mnews/article/001/001...,[\n휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대\n\n\n\n[연합뉴스 ...,2024-07-22 06:15:01
197,금융당국 채찍 드니까 PF 손실 인식… 대형 증권사도 2분기 어닝쇼크 가능성 커져,https://n.news.naver.com/mnews/article/366/000...,[\n6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기...,2024-07-22 06:01:22


In [58]:
df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [59]:
df

,news_title,news_url,news_contents,news_dates
0,줍줍까지 나왔는데 웃돈만 10억…'얼죽신' 열풍에 입주권 귀하신 몸,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 ...,2024-07-26 20:06:12
1,‘AI 반도체’ 리벨리온 IPO 주관사에 대표 삼성·공동 한투 선정,https://n.news.naver.com/mnews/article/366/000...,[\n기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 ...,2024-07-26 17:46:14
2,서울 세운5-1·3구역 대출 연장 성공…규모 1000억 확대,https://n.news.naver.com/mnews/article/018/000...,[\n대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에...,2024-07-26 19:48:07
3,"배임까지… 두산밥캣, 6거래일째 하락 이어 시간外 약세",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹...,2024-07-26 17:21:13
4,상승 출발 엔비디아 vs 하락 출발 테슬라 …과연 누가 웃을까?,https://n.news.naver.com/mnews/article/029/000...,[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자...,2024-07-26 22:56:07
...,...,...,...,...
194,"셀트리온, 하반기 짐펜트라 잠재력 확인-상상인",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반...,2024-07-22 05:09:12
195,"열기 빠진 시장…피앤에스미캐닉스 VS 아이빔테크놀로지, 증권가 ‘픽’은",https://n.news.naver.com/mnews/article/018/000...,"[\n[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호...",2024-07-22 00:01:09
196,5대 금융 트래블 체크카드 가입자 650만명 육박,https://n.news.naver.com/mnews/article/001/001...,[\n휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대\n\n\n\n[연합뉴스 ...,2024-07-22 06:15:01
197,금융당국 채찍 드니까 PF 손실 인식… 대형 증권사도 2분기 어닝쇼크 가능성 커져,https://n.news.naver.com/mnews/article/366/000...,[\n6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기...,2024-07-22 06:01:22


In [60]:
df.to_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_최종.csv", encoding='utf-8', index=False)